In [56]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Configuring

In [57]:
# URL base 
base_url = "https://www.zapimoveis.com.br/venda/apartamentos/mg+belo-horizonte/?itl_id=1000102&itl_name=zap_-_seo-widgets-group_home_static_to_zap_resultado-pesquisa" # Change

# Objeto geolocalizado
geolocator = Nominatim(user_agent="geoapi")

# Armazenar resultados
dados = []

# Número de páginas para scraping
num_paginas = 50     # Change

In [58]:
# Configurar o Selenium para rodar sem abrir a interface do navegador (modo headless)
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")  # Inicia maximizado
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Caminho para o ChromeDriver
chrome_driver_path = "C:\\Program Files\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

# Usando o Service para especificar o caminho do ChromeDriver
service = Service(chrome_driver_path)

# Criando o driver
driver = webdriver.Chrome(service=service, options=options)
driver.get(base_url)

# Espera explícita para garantir que a página seja carregada
wait = WebDriverWait(driver, 20)  # Espera por até 20 segundos
try:
    # Substitua 'id_do_elemento' pelo ID correto
    button = wait.until(EC.presence_of_element_located((By.ID, 'id_do_elemento')))
    button.click()
except Exception as e:
    print(f"Erro ao interagir com a página: {e}")
finally:
    driver.quit()

# Simulando o clique em um botão, caso o Cloudflare peça
button = driver.find_element(By.ID, "botao-captcha")  # Substitua pelo ID correto
button.click()

# Ou preencher um formulário de verificação, se necessário
input_field = driver.find_element(By.ID, "campo-de-entrada")
input_field.send_keys("resposta")

# Confirmar envio do formulário
submit_button = driver.find_element(By.ID, "botao-enviar")
submit_button.click()

# Espera explícita até que um elemento esteja presente na página
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "id-do-elemento"))
)

# Agora você pode interagir com o elemento encontrado
print(element.text)

Erro ao interagir com a página: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF7D6DFFB05+28789]
	(No symbol) [0x00007FF7D6D686E0]
	(No symbol) [0x00007FF7D6C0592A]
	(No symbol) [0x00007FF7D6BDF505]
	(No symbol) [0x00007FF7D6C86477]
	(No symbol) [0x00007FF7D6C9EF42]
	(No symbol) [0x00007FF7D6C7F1E3]
	(No symbol) [0x00007FF7D6C4A938]
	(No symbol) [0x00007FF7D6C4BAA1]
	GetHandleVerifier [0x00007FF7D713933D+3410093]
	GetHandleVerifier [0x00007FF7D714E7DD+3497293]
	GetHandleVerifier [0x00007FF7D7142A73+3448803]
	GetHandleVerifier [0x00007FF7D6EC7BBB+848171]
	(No symbol) [0x00007FF7D6D73C3F]
	(No symbol) [0x00007FF7D6D6F6E4]
	(No symbol) [0x00007FF7D6D6F87D]
	(No symbol) [0x00007FF7D6D5ED49]
	BaseThreadInitThunk [0x00007FFAE4E4259D+29]
	RtlUserThreadStart [0x00007FFAE6AAAF38+40]



KeyboardInterrupt: 

Scraping

In [38]:
for pagina in range(1, num_paginas + 1):
    url = f"{base_url}?pagina={pagina}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    imoveis = soup.find_all("div", class_="property-card__content")

    for imovel in imoveis:
        preco = imovel.find("div", class_="property-card__price").get_text(strip=True) if imovel.find("div", class_="property-card__price") else "N/A"
        descricao = imovel.find("span", class_="property-card__title").get_text(strip=True) if imovel.find("span", class_="property-card__title") else "N/A"
        area = imovel.find("span", class_="property-card__detail-area").get_text(strip=True) if imovel.find("span", class_="property-card__detail-area") else "N/A"
        quartos = imovel.find("li", class_="property-card__detail-room").get_text(strip=True) if imovel.find("li", class_="property-card__detail-room") else "N/A"
        banheiros = imovel.find("li", class_="property-card__detail-bathroom").get_text(strip=True) if imovel.find("li", class_="property-card__detail-bathroom") else "N/A"
        garagem = imovel.find("li", class_="property-card__detail-garage").get_text(strip=True) if imovel.find("li", class_="property-card__detail-garage") else "N/A"
        localizacao = imovel.find("span", class_="property-card__address").get_text(strip=True) if imovel.find("span", class_="property-card__address") else "N/A"
        imagem = imovel.find("img", class_="property-card__image")["src"] if imovel.find("img", class_="property-card__image") else "N/A"

        dados.append({
            "preco": preco,
            "descricao": descricao,
            "area": area,
            "quartos": quartos,
            "banheiros": banheiros,
            "garagem": garagem,
            "localizacao": localizacao,
            "imagem": imagem
        })

Converting to DataFrame

In [39]:
df = pd.DataFrame(dados)

Adding geolocation

In [40]:
def get_geocode(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        return None, None

if "localizacao" in df.columns:
    df["latitude"], df["longitude"] = zip(*df["localizacao"].apply(get_geocode))
else:
    print("A coluna 'localizacao' não foi encontrada no DataFrame.")

A coluna 'localizacao' não foi encontrada no DataFrame.


Cleaning data

In [41]:
if "preco" in df.columns:
    df["preco"] = (
        df["preco"]
        .str.replace(r"R\$|\.|,", "", regex=True) 
        .str.extract(r"(\d+)") 
        .astype(float)
    )
else:
    print("A coluna 'preco' não foi encontrada no DataFrame.")

if "area" in df.columns:
    df["area"] = df["area"].str.replace("m²", "").str.strip().astype(float)
else:
    print("A coluna 'area' não foi encontrada no DataFrame.")

if "quartos" in df.columns:
    df["quartos"] = df["quartos"].str.extract(r"(\d+)").astype(float)
else:
    print("A coluna 'quartos' não foi encontrada no DataFrame.")
    
if "banheiros" in df.columns:
    df["banheiros"] = df["banheiros"].str.extract(r"(\d+)").astype(float)
else:
    print("A coluna 'banheiros' não foi encontrada no DataFrame.")
    
if "garagem" in df.columns:
    df["garagem"] = df["garagem"].str.extract(r"(\d+)").astype(float)
else:
    print("A coluna 'garagem' não foi encontrada no DataFrame.")

A coluna 'preco' não foi encontrada no DataFrame.
A coluna 'area' não foi encontrada no DataFrame.
A coluna 'quartos' não foi encontrada no DataFrame.
A coluna 'banheiros' não foi encontrada no DataFrame.
A coluna 'garagem' não foi encontrada no DataFrame.


Saving in CSV

In [42]:
csv_name = "ZAP_imoveis_ap_bh" # Change

df.to_csv(f"../data/{csv_name}.csv", index=False)

print(f"Scraping concluído! {len(df)} imóveis coletados e geolocalizados.")

Scraping concluído! 0 imóveis coletados e geolocalizados.
